In [71]:
import torch
import random

In [72]:
# 设置原始数据
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.1, y.shape)
    return X, y.reshape(-1, 1)
true_w = torch.tensor([3, -2.5])
true_b = 3.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [73]:
# 提取小批量数据
def data_iter(batch_size, features, labels):
    num = len(features)
    indices = list(range(num))
    random.shuffle(indices)
    for i in range(0, num, batch_size):
        batch_indices = torch.tensor(indices[i : min(i + batch_size, num)])
        yield features[batch_indices], labels[batch_indices]

In [74]:
size = 10
for X, y in data_iter(size, features, labels):
    print(X, '\n', y)
    break

tensor([[-1.9534,  0.0582],
        [-0.3468,  0.7772],
        [-1.7711,  0.9482],
        [ 0.6433,  0.9330],
        [-0.7845, -1.3404],
        [-0.0105,  1.0506],
        [ 0.0575,  0.4204],
        [ 0.1202,  0.3753],
        [ 0.5710, -0.1636],
        [ 1.4053, -0.3704]]) 
 tensor([[-2.9013],
        [ 0.3934],
        [-4.5069],
        [ 2.7292],
        [ 4.1471],
        [ 0.6310],
        [ 2.3402],
        [ 2.7832],
        [ 5.3822],
        [ 8.3083]])


In [75]:
# 设定模型与损失函数
def Linear(X, w, b):
    return torch.matmul(X, w) + b
def loss_squared(y_hat, y_true):
    return (y_hat - y_true.reshape(y_hat.shape)) ** 2 / 2

In [76]:
# 设定优化算法
def sgd(params, learn_rate, batch_size):
    with torch.no_grad():
        for param in params:
            param -= learn_rate * param.grad / batch_size
            param.grad.zero_()

In [77]:
# 初始化参数
w = torch.normal(0, 1, size=(2,), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [78]:
# 设置超参数
learn_rate = 0.03
num_eqochs = 6
batch_size = 10
net = Linear
loss = loss_squared

In [79]:
# 训练
for eqoch in range(num_eqochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        l.sum().backward()
        sgd([w, b], learn_rate, batch_size)

In [80]:
print(true_w - w.reshape(true_w.shape))
print(true_b - b)

tensor([0.0003, 0.0045], grad_fn=<SubBackward0>)
tensor([0.0011], grad_fn=<RsubBackward1>)
